# Setup and Load dataset



In [ ]:
%%capture
!pip install elasticsearch==7.14.0
!apt install default-jdk > /dev/null

In [ ]:
!pip install sentence_transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 65.7 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=b40624a7b7b7eff6d30a98d2c5d3d6abae0972340959adbd22fdef1d13406adf
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence_transformers


In [ ]:
!pip install --upgrade --no-cache-dir gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


In [ ]:
!gdown --id 1pxg131HVz6t1lGPDiT70SIf8LnVJlVy4

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1pxg131HVz6t1lGPDiT70SIf8LnVJlVy4
To: /content/medium_data.zip
100% 520k/520k [00:00<00:00, 159MB/s]


In [ ]:
!unzip -o "medium_data.zip"  -d  "/content"

Archive:  medium_data.zip
  inflating: /content/medium_data.csv  


In [ ]:
import os
os.kill(os.getpid(), 9)

## Data Loading

In [ ]:
from datetime import datetime
import os
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from scipy import sparse
from scipy.sparse import csc_matrix
from sklearn.decomposition import TruncatedSVD

np.random.seed(0)

#### Name of the file which contain all the item properties

In [ ]:
file="medium_data.csv"

###### Run below cell

In [ ]:
content_df  = pd.read_table( file,delimiter=',')

In [ ]:
content_df.head(1)

,id,url,title,subtitle,image,claps,responses,reading_time,publication,date
0,1,https://towardsdatascience.com/a-beginners-gui...,A Beginner’s Guide to Word Embedding with Gens...,NaN,1.png,850,8,8,Towards Data Science,2019-05-30


## Details about dataset

In [ ]:
itemid="id"

In [ ]:
features=['title','subtitle'	]

In [ ]:
allcols=[itemid]
for i in features:
  allcols.append(i)

# Setup

In [ ]:
content_df['NewTag']=""
for i in features:
  content_df[i] = content_df[i].fillna(' ')
for i in features:
  content_df[i] = content_df[i].fillna(' ')
for i in features:
  content_df['NewTag']+=(' '+content_df[i])
content_df['NewTag']=content_df['NewTag'].astype(str)

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

In [ ]:
def clean_text(text):
    
    text = text.lower()  # lowercase text
    # replace the matched string with ' '
    text = re.sub( re.compile("\'s"), ' ', text)
    text = re.sub(re.compile("\\r\\n"), ' ', text)
    text = re.sub(re.compile(r"[^\w\s]"), ' ', text)
    return text

In [ ]:
stopwords=set(stopwords.words('english'))

In [ ]:
def tokenizer(sentence, min_words=4, max_words=200, stopwords=stopwords, lemmatize=True):
    
    if lemmatize:
        stemmer = WordNetLemmatizer()
        tokens = [stemmer.lemmatize(w) for w in word_tokenize(sentence)]
    else:
        tokens = [w for w in word_tokenize(sentence)]
    token = [w for w in tokens if (len(w) > min_words and len(w) < max_words
                                                        and w not in stopwords)]
    return tokens    

In [ ]:
content_df['clean'] = content_df['NewTag'].apply(clean_text)
# content_df['token_lem_sentence'] = content_df['clean'].apply(
#         lambda x: tokenizer(x))

# Elastic search set up

In [ ]:
import elasticsearch
from elasticsearch import Elasticsearch
from elasticsearch import helpers


In [ ]:
# Download & extract Elasticsearch 7.0.0

!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.0.0-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.0.0-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.0.0

In [ ]:
# Creating daemon instance of elasticsearch
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.0.0/bin/elasticsearch'], 
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                 )

In [ ]:
# This part is important, since it takes a little amount of time for instance to load
import time
time.sleep(20)

In [ ]:
%%bash
# If you get 1 root & 2 daemon process then Elasticsearch instance has started successfully
ps -ef | grep elasticsearch

daemon      1867    1677 99 04:29 ?        00:00:21 /content/elasticsearch-7.0.0/jdk/bin/java -Xms1g -Xmx1g -XX:+UseConcMarkSweepGC -XX:CMSInitiatingOccupancyFraction=75 -XX:+UseCMSInitiatingOccupancyOnly -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Djava.io.tmpdir=/tmp/elasticsearch-11574500148561208233 -XX:+HeapDumpOnOutOfMemoryError -XX:HeapDumpPath=data -XX:ErrorFile=logs/hs_err_pid%p.log -Xlog:gc*,gc+age=trace,safepoint:file=logs/gc.log:utctime,pid,tags:filecount=32,filesize=64m -Djava.locale.providers=COMPAT -Dio.netty.allocator.type=unpooled -Des.path.home=/content/elasticsearch-7.0.0 -Des.path.conf=/content/elasticsearch-7.0.0/config -Des.distribution.flavor=default

In [ ]:
# Check if elasticsearch is running
!curl -sX GET "localhost:9200/"

In [ ]:
es = Elasticsearch(hosts = [{"host":"localhost", "port":9200}])
# Check if python is connected to elasticsearch
es.ping()

True

# Search BM25

In [ ]:
# Define settings & mappings of Elasticsearch index
Settings = {
    "settings":{
        "number_of_shards":1,
        "number_of_replicas":0
    },
    "mappings":{
        "properties":{
            "id":{
                "type":"text"
            },
            "longtext":{
                "type":"text"
            }
        }
    }
}

In [ ]:
indexname='new_index1'
index_type='_doc'

In [ ]:

index = es.indices.create(index=indexname, ignore=[400,404], body=Settings)


In [ ]:
json_list = []

for idx, row in content_df.iterrows():
            dic = {}
            dic['_index'] = indexname
            dic['_type'] = index_type
            source = {}
            source[itemid] = row[itemid]
            source['longtext'] = row['clean']
            dic['_source'] = source
            json_list.append(dic)

In [ ]:
# For importing Data to elasticsearch we use elasticsearch's bulk API from elasticsearch.helpers
try:
    res = helpers.bulk(es, json_list)
    print("successfully imported to elasticsearch.")
except Exception as e:
    print(f"error: {e}")

/usr/local/lib/python3.8/dist-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


successfully imported to elasticsearch.


In [ ]:
search_query="graph neural network"

In [ ]:
# Complicated query
query = es.search(
    index=indexname,
    body={
        "size":20,
        "query":{
            "bool":{
                "must":[
                        {"match":{"longtext":search_query}}
                ],
                # "should":[
                #         {"match":{"":""}}
                # ]
            }
        }
    }
)

output = pd.json_normalize((query['hits']['hits']))
output

,_index,_type,_id,_score,_source.id,_source.longtext
0,new_index1,_doc,JbUWSYYBHhi0keru38uW,11.091132,5860,sep how to build a simple neural network from...
1,new_index1,_doc,rbUWSYYBHhi0keru2775,10.660841,2668,sep an introduction to convolutional neural n...
2,new_index1,_doc,IbUWSYYBHhi0keru2ree,10.546149,736,sep neural networks training with approximate...
3,new_index1,_doc,FrUWSYYBHhi0keru2ree,9.416538,725,sep exploring how neural networks work and ma...
4,new_index1,_doc,hbUWSYYBHhi0keru3cbZ,9.359104,4676,sep practical graph neural networks for molec...
5,new_index1,_doc,UrUWSYYBHhi0keru4Mwo,9.144320,6161,sep neural network optimization sep covering ...
6,new_index1,_doc,ZrUWSYYBHhi0keru27oa,8.816080,1573,sep kohonen self organizing maps sep a specia...
7,new_index1,_doc,KrUWSYYBHhi0keru38uW,8.816080,5865,sep finding the right architecture for neural...
8,new_index1,_doc,JrUWSYYBHhi0keru3cfZ,8.771564,4837,sep evolution of graph neural networks for re...
9,new_index1,_doc,zrUWSYYBHhi0keru2rjY,8.510588,1165,sep is relu reluvant sep questioning very ba...
